In [1]:
# !pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip install -q datasets loralib sentencepiece 
# !pip -q install bitsandbytes accelerate
# !pip -q install langchain

### We can use LangChain - HuggingFacePipeline to wrap the custom LLMs. but this only support text2text-generation, text-generation pipelines

In [11]:
import torch, warnings
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain import PromptTemplate, LLMChain
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
warnings.filterwarnings("ignore")

#### 1. download alpaca natives

In [2]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model_card = "chavinlo/alpaca-native"
device_map = 'auto'

tokenizer = LlamaTokenizer.from_pretrained(model_card)
base_model = LlamaForCausalLM.from_pretrained(
                                            model_card,
                                            quantization_config=quantization_config,
                                            device_map=device_map
                                            )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#### 2. build the pipeline

In [3]:
pipe = pipeline(
                "text-generation",
                model=base_model, 
                tokenizer=tokenizer, 
                max_length=256,
                temperature=0.6,
                top_p=0.95,
                repetition_penalty=1.2
                )

local_llm = HuggingFacePipeline(pipeline=pipe)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


#### 3. build the prompt

In [7]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(
                        template=template, 
                        input_variables=["instruction"]
                        )

In [8]:
llm_chain = LLMChain(
                     prompt=prompt, 
                     llm=local_llm
                     )

#### 3. experiment on samples

In [9]:
question = "What is the capital of England?"
print(llm_chain.run(question))

 The capital of England is London.


In [10]:
question = "What are alpacas? and how are they different from llamas?"

print(llm_chain.run(question))

 Alpacas are small, domesticated animals related to camels. They are native to Peru and Bolivia, where they have been used as pack animals for carrying goods on mountain trails. Llamas are also small, but much more common, and are native to South America. Unlike alpacas, they are not domesticated domesticate and do not make good pack animals because of their size.


#### 4. setting up a chat with memory

In [12]:
# use window memory approach & set window size to 4
window_memory = ConversationBufferWindowMemory(k=4)

conversation = ConversationChain(
                                llm=local_llm, 
                                verbose=True, 
                                memory=window_memory
                                )

In [14]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [15]:
# overwrite the original prompt template

conversation.prompt.template = '''The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
{history}
Human: {input}
AI:'''

In [16]:
conversation.predict(input="Hi there! I am Isuru")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:

Human: Hi there! I am Isuru
AI:

> Finished chain.


" Hey there! It's nice to meet you too. What can I help you with?"

In [17]:
conversation.predict(input="What is your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
Human: Hi there! I am Isuru
AI:  Hey there! It's nice to meet you too. What can I help you with?
Human: What is your name?
AI:

> Finished chain.


' My name is Alpaca. How did you come up with that name?'

In [18]:
conversation.predict(input="Can you tell me what an Alpaca is?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
Human: Hi there! I am Isuru
AI:  Hey there! It's nice to meet you too. What can I help you with?
Human: What is your name?
AI:  My name is Alpaca. How did you come up with that name?
Human: Can you tell me what an Alpaca is?
AI:

> Finished chain.


' Sure. An alpaca is a species of South American camelid mammal. They are typically found in the Andes mountains and live in herds led by an alpha female leader.'

In [19]:
conversation.predict(input="How is it different from a Llama?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
Human: Hi there! I am Isuru
AI:  Hey there! It's nice to meet you too. What can I help you with?
Human: What is your name?
AI:  My name is Alpaca. How did you come up with that name?
Human: Can you tell me what an Alpaca is?
AI:  Sure. An alpaca is a species of South American camelid mammal. They are typically found in the Andes mountains and live in herds led by an alpha female leader.
Human: How is it different from a Llama?
AI:

> Finished chain.


'  Alpacas and llamas belong to the same family but they have some differences as well. Alpacas tend to be smaller than llamas and they lack the long ear frills that llamas have. Additionally, their fleece is finer'

In [20]:
conversation.predict(input="Can you give me some good names for a pet llama?")

Input length of input_ids is 276, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
Human: Hi there! I am Isuru
AI:  Hey there! It's nice to meet you too. What can I help you with?
Human: What is your name?
AI:  My name is Alpaca. How did you come up with that name?
Human: Can you tell me what an Alpaca is?
AI:  Sure. An alpaca is a species of South American camelid mammal. They are typically found in the Andes mountains and live in herds led by an alpha female leader.
Human: How is it different from a Llama?
AI:   Alpacas and llamas belong to the same family but they have some differences as well. Alpacas tend to be smaller than llamas and they lack the long ear frills that llamas have. Additionally, their fleece is finer
Human: Can y

' '

In [21]:
conversation.predict(input="Is your name LLaMA?")

Input length of input_ids is 257, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
Human: What is your name?
AI:  My name is Alpaca. How did you come up with that name?
Human: Can you tell me what an Alpaca is?
AI:  Sure. An alpaca is a species of South American camelid mammal. They are typically found in the Andes mountains and live in herds led by an alpha female leader.
Human: How is it different from a Llama?
AI:   Alpacas and llamas belong to the same family but they have some differences as well. Alpacas tend to be smaller than llamas and they lack the long ear frills that llamas have. Additionally, their fleece is finer
Human: Can you give me some good names for a pet llama?
AI:  
Human: Is your name LLaMA?
AI:

> Finished chai

' No'

In [22]:
conversation.predict(input="What food should I feed my new llama?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI called Alpaca. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. 
 
Current conversation:
Human: Can you tell me what an Alpaca is?
AI:  Sure. An alpaca is a species of South American camelid mammal. They are typically found in the Andes mountains and live in herds led by an alpha female leader.
Human: How is it different from a Llama?
AI:   Alpacas and llamas belong to the same family but they have some differences as well. Alpacas tend to be smaller than llamas and they lack the long ear frills that llamas have. Additionally, their fleece is finer
Human: Can you give me some good names for a pet llama?
AI:  
Human: Is your name LLaMA?
AI:  No
Human: What food should I feed my new llama?
AI:

> Finished chain.


'  Your new llama should eat grass,'